# 뉴스기사 분류모델
## 1. 형태소 분석엔진 설치 (Colab)

In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
!bash Mecab-ko-for-Google-Colab/install_mecab-ko_on_colab_light_220429.sh

## 2. 패키지 참조
### 1) helper 참조

In [ ]:
import sys
# sys.path.append('../../')
import helper

### 2) 다른 패키지들 참조

In [ ]:
import os
import numpy as np
import seaborn as sb
import requests

from pandas import DataFrame, read_excel
from konlpy.tag import Mecab
from matplotlib import pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embeddint, Dense, GRU
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequences import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.utils import to_categorical

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

## 3. 데이터셋 준비하기
### 1) 뉴스기사 데이터셋

In [ ]:
origin = read_excel('./news.xlsx')
origin.head()

### 2) 불어용 데이터셋

In [ ]:
r = requests.get('https://data.hossam/kr/korean_stopwords.txt')
r.encoding='utf-8'
stopwords = r.text.split('\n')
print(stopwords)

## 4. 데이터 전처리
### 1) 뉴스기사에서 영어, 특수문자를 제거하고 한글만 남기기

In [ ]:
df = origin.copy()

# 한글을 제외한 나머지 글자들을 빈 문자열로 대체
df['content'] = df['content'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]', '', regex=True)

# document 컬럼의 데이터들 중에서 빈 문자열만 존재하는 항목은 결측치로 대체
df['content'].replace('', np.nan, inplace=True)

# 전체 데이터 셋 크기 확인
print('데이터 크기: ', df['content'].shape)

# 결측치 확인
print('결측치 크기: ', df['content'].isna().sum())

### 2) 결측치 제거

In [1]:
df.dropna(inplace=True)
df.isna().sum()

NameError: name 'df' is not defined

### 3) 종속 변수 라벨링
#### 종속변수 값 확인 및 종류를 딕셔너리로 변환

In [2]:
# 종속 변수 값의 종류
category = list(df['category].unique())
category

# 종속 변수 값의 종류를 딕셔너리로 변환
cat_dict = {}
for i, v in enumerate(category):
    cat_dict[v] = i
cat_dict

SyntaxError: EOL while scanning string literal (4234776867.py, line 2)

#### 종속 변수 라벨링

In [ ]:
df['category'] = df['category'].map(cat_dict)
df['category'].value_counts()

### 4) 뉴스 기사 형태소 분석

In [ ]:
if sys.platform == 'win32':
    mecab = Mecab(dicpath="C:\\mecab\\mecab-ko-dic")
else:
    mecab = Mecab()

# 문장내 형태소들을 저장할 리스트
word_set = []

# 덧글 내용에 대해 반복 처리
for i, v in enumerate(df['content']):
    # 덧글 하나에 대한 형태소 분석
    morphs = mecab.morphs(v)
    # print(morphs)
    # if i > 5:
    #     break

    # 형태소 분석 결과에서 불용어를 제외한 단어만 별도의 리스트로 생성
    confirm_words = []
    for j in morphs:
        if j not in stopwords:
            confirm_words.append(j)

    # 불용어를 제외한 형태소 리스트를 통째로 word_set에 저장함
    # -> word_set은 2차원 리스트가 된다. 1차원이 덧글 단위임
    word_set.append(confirm_words)

# 상위 3건만 출력해서 확인
word_set[:3]

### 5) 형태소 토큰화
#### 전체 단어에 대한 토큰화

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(word_set)
print(f'전체 단어수: {len(tokenizer.word_index)}')

#### 3회 이상 자주 등장하는 단어의 수 구하기

In [3]:
# 사용 빈도가 높다고 판단할 등장 회수
threshold = 3

# 전체 단어의 수
total_cnt = len(tokenizer.word_index)

# 등장 빈도수가 threshold보다 작은 단어의 개수를 카운트할 값
rare_cnt = 0

# 훈련 데이터의 전체 단어 빈도수 총 합
total_freq = 0

# 등장 빈도수가 threshold보다 작은 단어의 등장 빈도수의 총 합
rare_freq = 0

# 단어와 빈도수의 쌍(pair)을 key와 value로 받는다.
for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    # 단어의 등장 빈도수가 threshold보다 작으면
    if(value < threshold):
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print('단어 집합(vocabulary)의 크기 :',total_cnt)
print('등장 빈도가 %s번 미만인 희귀 단어의 수: %s' % (threshold, rare_cnt))
print("단어 집합에서 희귀 단어의 비율:", (rare_cnt / total_cnt)*100)
print("전체 등장 빈도에서 희귀 단어 등장 빈도 비율:", (rare_freq / total_freq)*100)

# 자주 등장하는 단어 집합의 크기 구하기 -> 이 값이 첫 번째 학습층의 input 수가 된다.
vocab_size = total_cnt - rare_cnt + 2
print('단어 집합의 크기 :', vocab_size)

NameError: name 'tokenizer' is not defined